In [171]:
import json
from collections import Counter
from dataclasses import asdict
from functools import reduce, partial
from typing import List, Set

import numpy as np
from more_itertools import minmax
from nltk import word_tokenize
from scipy.spatial.distance import cosine
from scipy.special import softmax
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import normalize
from tqdm import tqdm

from src.clustering import embeddings
from src.clustering.embeddings.params import Params
from src.clustering.embeddings.tokenizer import Tokenizer
from src.clustering.keyword import KeywordGroup
from src.settings import DATA_PATH, MODELS_PATH
from src.utils import file_paths
from src.utils.merge_csv import merge

In [172]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [173]:
model_params = dict(
    eps=0.08,
    min_samples=2,
    metric='cosine'
)

In [174]:
params = Params(
    v='1.8.5',
    group_fn='weighted sum',
    model='DBSCAN',
    model_params=model_params
)

In [175]:
df = merge(file_paths(DATA_PATH / 'raw' / 'data_validation'), columns_take=3)

In [176]:
keywords: List[Set[str]] = (
    df
        .keywords_processed
        .dropna()
        .map(lambda s: s.split(','))
        .tolist()
)

In [177]:
tokenizer = Tokenizer(
    embeddings
        .get_source('ruwiki')
        .load(MODELS_PATH / 'embeddings')
)

[INFO] loading projection weights from D:\projects\rpd-keyword-extraction\models\embeddings\ruwikiruscorpora_upos_cbow_300_10_2021\model.bin
[INFO] KeyedVectors lifecycle event {'msg': 'loaded (249333, 300) matrix of type float32 from D:\\projects\\rpd-keyword-extraction\\models\\embeddings\\ruwikiruscorpora_upos_cbow_300_10_2021\\model.bin', 'binary': True, 'encoding': 'utf8', 'datetime': '2022-03-29T19:42:29.003790', 'gensim': '4.1.2', 'python': '3.9.6 (tags/v3.9.6:db3ff76, Jun 28 2021, 15:26:21) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22000-SP0', 'event': 'load_word2vec_format'}
c:\users\uiqko\pyenvs\mint\lib\site-packages\gensim\models\keyedvectors.py:478: UserWarning: Adding single vectors to a KeyedVectors which grows by one each time can be costly. Consider adding in batches or preallocating to the required size.
  warnings.warn(



In [178]:
all_keywords = set(map(str.strip, reduce(list.__add__, keywords)))

In [179]:
counter = Counter(
    reduce(
        list.__add__,
        map(tokenizer.tokenize, all_keywords)
    )
)
counter[tokenizer.default_index] = 1

In [180]:
all_keywords = set(all_keywords)
# todo??/
all_keywords.remove('')
all_keywords.remove(' ')

In [181]:
def keyword_to_embeddings(keyword: str):
    embeddings = tokenizer.vectorize(
        tokenizer.tokenize(keyword), to=np.array)

    return embeddings

In [182]:
mn, mx = minmax(counter.values())
m = mx - mn

def normalize_(x):
    return np.array([(x_ - mn) / m for x_ in x])

In [183]:
def keyword_to_embedding(keyword: str):  # 'w1 w2 ... wn'
    indexes = tokenizer.tokenize(keyword)
    counts = list(map(counter.get, indexes))
    # vec = 1 - softmax(normalize([counts]))[0]
    vec = 1 - normalize_(counts)
    embeddings = keyword_to_embeddings(keyword)  # n x 300

    return embeddings.T @ vec  # 300 x 1


In [184]:
model = DBSCAN(**model_params)

In [185]:
keyword_by_embedding = {}
embedding_by_keyword = {}
X = []

for keyword in tqdm(all_keywords, desc='Embeddings'):
    embedding = keyword_to_embedding(keyword)
    embedding_key = tuple(embedding)
    # todo +0.001 if key in dict
    keyword_by_embedding[embedding_key] = keyword
    embedding_by_keyword[keyword] = embedding
    X.append(embedding)

model.fit_predict(X)


Embeddings: 100%|██████████| 11301/11301 [00:11<00:00, 945.69it/s]


array([ -1,  -1,   0, ...,  -1,  -1, 699], dtype=int64)

In [186]:
groups = {}
other = []

for i, label in enumerate(model.labels_):
    if label == -1:
        other.append(keyword_by_embedding[tuple(X[i])])
    else:
        groups.setdefault(label, set())\
            .add(keyword_by_embedding[tuple(X[i])])

In [187]:
def keyword_group_center(kws: List[str]) -> str:
    keyword_embeddings = list(map(embedding_by_keyword.get, kws))
    center = np.mean(keyword_embeddings, axis=0)
    nearest = min(keyword_embeddings, key=partial(cosine, center))
    return keyword_by_embedding[tuple(nearest)]

In [188]:
def most_common_group_center(kws: List[str]) -> str:
    return ', '.join(dict(Counter(word_tokenize(' '.join(kws))).most_common(2)).keys())

In [189]:
keyword_groups = [
    KeywordGroup(most_common_group_center(kws), kws)
    for kws in groups.values()
]

In [190]:
keyword_groups

[KeywordGroup(mean='полевые, транзисторы', keywords={'полевые транзисторы'}),
 KeywordGroup(mean='переменные', keywords={'переменные'}),
 KeywordGroup(mean='линеаризация, динамической', keywords={'линеаризация динамической модели электропривода', ' линеаризация динамической модели'}),
 KeywordGroup(mean='действия, принципы', keywords={' принципы действия устройств', 'физические принципы действия', ' принцип действия', ' принципы действия оборудования', ' принцип действия преобразователей', ' физические принципы действия устройств', ' принцип действия приборов'}),
 KeywordGroup(mean='maps', keywords={' maps'}),
 KeywordGroup(mean='скорость, распространения', keywords={'скорость распространения'}),
 KeywordGroup(mean='материалов, свойства', keywords={' анализ свойств материалов', ' исследования теплофизических свойств материалов', 'расчет термодинамическим методом', ' термодинамические методы расчета свойств материалов', ' физикохимические свойства материала', 'свойства полимерных матери

In [191]:
print(groups)

{0: {'полевые транзисторы'}, 1: {'переменные'}, 2: {'линеаризация динамической модели электропривода', ' линеаризация динамической модели'}, 3: {' принципы действия устройств', 'физические принципы действия', ' принцип действия', ' принципы действия оборудования', ' принцип действия преобразователей', ' физические принципы действия устройств', ' принцип действия приборов'}, 4: {' maps'}, 5: {'скорость распространения'}, 6: {' анализ свойств материалов', ' исследования теплофизических свойств материалов', 'расчет термодинамическим методом', ' термодинамические методы расчета свойств материалов', ' физикохимические свойства материала', 'свойства полимерных материалов', ' теплофизические свойства материалов', 'методы расчета свойств материалов', ' свойства материалов', ' свойства пищевых материалов', ' термодинамические методы расчета свойств', ' свойства конструкционных материалов'}, 7: {'виртуальная реальность', 'технологии виртуальной реальности'}, 8: {' износостойкость'}, 9: {'теплофи

In [192]:
fname = f'clusters' \
            f'.v{params.v}' \
            f'.size_{len(df)}' \
            f'.{params.model}' \
            f'.json'

In [193]:
with open(DATA_PATH / 'results' / 'clusters' / fname, 'w', encoding='utf-8') as f:
    json.dump({
        'params': asdict(params),
        'clusters': [
            {
                'mean': g.mean,
                'keywords': list(g.keywords)
            } for g in keyword_groups
        ]
    }, f, indent=4, ensure_ascii=False)

## ...

In [194]:
[
    "пошение качества разработки",
    "качество технологий",
    "качество проекта",
    "оценка качества",
    "качество изображения",
    "качество системы",
    "качества безопасности"
]

['пошение качества разработки',
 'качество технологий',
 'качество проекта',
 'оценка качества',
 'качество изображения',
 'качество системы',
 'качества безопасности']

In [195]:
counter[tokenizer.single_tokenize('качество')]

72

In [196]:
counter[tokenizer.single_tokenize('безопасности')]

112

In [197]:
['оценка качества', 'качество проекта', 'качества безопасности']

['оценка качества', 'качество проекта', 'качества безопасности']

In [198]:
1 - softmax(list(map(counter.get, tokenizer.tokenize("качества безопасности"))))

array([1., 0.])

In [199]:
tokenizer.tokenize("качества безопасности")

[166, 979]

In [200]:
counter[tokenizer.single_tokenize('экспоненциальные')]

2

In [201]:
counter[tokenizer.single_tokenize('методы')]

398

In [202]:
list(map(counter.get, tokenizer.tokenize("теория баз данных")))

[129, 50, 192]

In [203]:
list(map(counter.get, tokenizer.tokenize("базы данных")))

[50, 192]

In [204]:
softmax(normalize([[2, 398]])[0])

array([0.26993304, 0.73006696])

In [205]:
cosine(keyword_to_embedding("методы измерений",), keyword_to_embedding("теплофизические измерения"))

0.10602856682146211

In [206]:
softmax(_)

1.0

In [207]:
keyword = "теплофизические измерения"
indexes = tokenizer.tokenize(keyword)
counts = list(map(counter.get, indexes))
vec = 1 - softmax(normalize([counts]))[0]
vec

array([0.65047708, 0.34952292])

In [208]:
normalize_(map(counter.get, tokenizer.tokenize("теплофизические измерения")))

array([0.03691275, 0.11073826])

In [209]:
keyword = "экспоненциальные методы анализа"
indexes = tokenizer.tokenize(keyword)
counts = list(map(counter.get, indexes))
vec = 1 - softmax(normalize([counts]))[0]
vec

array([0.80113259, 0.55386276, 0.64500465])

In [210]:
keyword = 'экспоненциальные методы анализа'
indexes = tokenizer.tokenize(keyword)
counts = list(map(counter.get, indexes))
# vec = 1 - softmax(normalize([counts]))[0]
vec = 1 - normalize_(counts)
vec

array([0.99832215, 0.33389262, 0.52181208])

In [211]:
indexes

[35288, 550, 1170]

In [212]:
tokenizer.default_index

249333

In [213]:
tokenizer.single_tokenize('база')

600

In [214]:
mn, mx = minmax(counter.values())
m = mx - mn

In [215]:
def normalize_(x):
    return np.array([max(mn, x_ - mn) / m for x_ in x])

In [216]:
normalize_([2, 398])

array([0.00167785, 0.66610738])

In [217]:
sum(counter.values())

24785

In [218]:
normalize([normalize_(map(counter.get, tokenizer.tokenize("теплофизические измерения")))], norm='max')

array([[0.33333333, 1.        ]])

In [219]:
tokenizer.single_tokenize('теплофизические')

84843